# Stochastic semi-nonparametric envelopment of z variables data (StoNEZD)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 16, 2020

References:

[1] Saastamoinen, A. and Kuosmanen, T. (2016). Quality frontier of electricity distribution: Supply security, best practices, and underground cabling in Finland, Energy Economics. 53, pp. 281–292.

[2] Kuosmanen, T., Saastamoinen, A. and Sipiläinen, T. What is the best practice for benchmark regulation of electricity distribution? Comparison of DEA, SFA and StoNED methods, Energy Policy. 61, pp. 740–750.

### Objective: Estimating a log-transformed cost function model with z-variable and technicial inefficiency

In [1]:
# import package pystoned
from pystoned import *

import pandas as pd
import numpy as np

### Import data

In [2]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [3]:
# output (total cost)
y  = df['TOTEX']

# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)

In [4]:
# Z variables
z = df['PerUndGr']

### Stage 1: Convex Nonparametric Least Squares estimation

In [6]:
# import the CNLSZ module
cet = "mult"
fun = "cost"
rts = "crs"

model = CNLSZ.cnlsz(y, x, z, cet, fun, rts)

In [7]:
# slove the model using remote solver (NEOS)
from pyomo.environ import SolverManagerFactory
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

In [8]:
# display estimates
model.e.display() 

e : residuals
    Size=89, Index=i
    Key : Lower : Value                   : Upper : Fixed : Stale : Domain
      0 :  None :     0.06655979752215041 :  None : False : False :  Reals
      1 :  None :    0.026027214472251904 :  None : False : False :  Reals
      2 :  None :     0.04364802505139959 :  None : False : False :  Reals
      3 :  None :   -0.008867996045194762 :  None : False : False :  Reals
      4 :  None :     0.18923050328904567 :  None : False : False :  Reals
      5 :  None :    -0.11484870791999217 :  None : False : False :  Reals
      6 :  None :   -0.015960383180604987 :  None : False : False :  Reals
      7 :  None :     0.15712232865530068 :  None : False : False :  Reals
      8 :  None :     0.22759422615991093 :  None : False : False :  Reals
      9 :  None :    -0.08712595244561908 :  None : False : False :  Reals
     10 :  None :     0.03445215153005295 :  None : False : False :  Reals
     11 :  None :    -0.07715596598915352 :  None : False : False

In [9]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)

### Stage 2: Residuals decomposition

In [10]:
# decomposition by method od moment (MoM)
method      = 'MoM'
res         = StoNED.stoned(y, eps, fun, method, cet)

In [11]:
# display technicial inefficiency
TE = res[1]
TE

array([0.87704436, 0.89270115, 0.88610556, 0.90475796, 0.82141804,
       0.9332887 , 0.90704258, 0.83681801, 0.80268988, 0.92693612,
       0.88958922, 0.92446933, 0.8073926 , 0.80395874, 0.91957403,
       0.93047728, 0.83867265, 0.78933311, 0.89619163, 0.92894328,
       0.92249804, 0.95858946, 0.78330233, 0.93173208, 0.83772132,
       0.89497657, 0.80144293, 0.94841837, 0.82348559, 0.90917094,
       0.91740745, 0.96045055, 0.91242742, 0.80193951, 0.90918372,
       0.83880878, 0.94943583, 0.94591177, 0.9382895 , 0.90603195,
       0.9019538 , 0.8997002 , 0.85681879, 0.93078024, 0.93381907,
       0.9378501 , 0.88759878, 0.86160469, 0.93791444, 0.9421487 ,
       0.79580054, 0.88994634, 0.84728921, 0.8135383 , 0.87578412,
       0.9517148 , 0.94299542, 0.91598328, 0.94878737, 0.91232394,
       0.94014509, 0.92013197, 0.91664729, 0.88497394, 0.77461079,
       0.92538884, 0.85017172, 0.92390857, 0.88603485, 0.95882326,
       0.91050437, 0.89170665, 0.93900162, 0.92704804, 0.93567